In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import requests
def get_reuters_articles(URL):

    chrome_options = Options()
    #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(URL)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    articles=soup.find_all("div", class_=lambda c: c and c.startswith("story-card__area-headline"))
    news_data = []
    for article in articles:
        title = article.get_text(strip=True)
        link = article.find('a', href=True)['href']
        if not link.startswith('http'):
            link = base_URL + link
        data_point = {'offset': offset_nb,'section':section_val,'Date': title, 'link': link}
        news_data.append(data_point)
    return pd.DataFrame(news_data)

get_article_sets =set()
for section_val in ['markets']:
    offset_nb = 0
    base_URL="https://www.reuters.com"
    search_query="/site-search/?query=gold"
    offset =f"&offset={offset_nb}"
    section=f"&section={section_val}"
    URL = base_URL + search_query + offset + section
    try:
        df = get_reuters_articles(URL)
        get_article_sets.add((section_val ,offset_nb))
    except Exception as e:
        print(f"Error fetching articles from {URL}: {e}")
    for offset_nb in range(20, 1000, 20):
        base_URL="https://www.reuters.com"
        search_query="/site-search/?query=gold"
        offset =f"&offset={offset_nb}"
        section=f"&section={section_val}"
        URL = base_URL + search_query + offset + section
        try:
            df_latest=get_reuters_articles(URL)
            df = pd.concat([df, df_latest], ignore_index=True)
            get_article_sets.add((section_val ,offset_nb))
        except Exception as e:
            print(f"Error fetching articles from {URL}: {e}")
            continue